# More Reading Material

* [The Math of Machine Learning](https://gwthomas.github.io/docs/math4ml.pdf)

# Taking Symbolic Derivatives with Python

In [ ]:
from sympy import *
from IPython.display import display
from sympy.printing.mathml import mathml
from IPython.display import display, Math, Latex

x, y, z = symbols('x y z')
init_printing(use_unicode=True)

In [ ]:
def mprint(e):
    display(Math(latex(e)))

In [ ]:
mprint(x**2)

In [ ]:
mprint(diff(2*x**3))

In [ ]:
expr = (x**3 + x**2 - x - 1)/(x**2 + 2*x + 1)
display(Math(latex(expr)))
expr = simplify(expr)
print(type(expr))
print(latex(expr))
display(Math(latex(expr)))

In [ ]:
from IPython.display import display, Math, Latex
display(Math('\\frac{1}{2}'))

In [ ]:
print(expr.subs(x,5))

In [ ]:
eql = Eq(3*x+5,10)

In [ ]:
z = solveset(eql,x)
display(Math(latex(z)))

In [ ]:
from sympy import *
x, y, z = symbols('x y z')
init_printing(use_unicode=True)
expr = diff(sin(x)/x**2, x)
mprint(expr)

In [ ]:
expr_i = integrate(expr,x)
mprint(expr_i)

# Keras Customization: Loss and Activation Functions

Your functions must be defined with TensorFlow graph commands.  The derivative will be taken automatically. (assuming all components of your function are differentiable)

# TensorFlow for Calculation

In [ ]:
import tensorflow as tf

tf.multiply(tf.constant(2),tf.constant(5))

In [ ]:
import numpy as np
tf.multiply(np.array([2,4]),np.array([2,4]))

In [ ]:
tf.multiply(2.0,4.0)

In [ ]:
tf.divide(2,4)

In [ ]:
tf.pow(2,4)

In [ ]:
x = 5.0
y = tf.divide(1.0,tf.add(1,tf.exp(tf.negative(x))))
y

# Calculus with TensorFlow

How do we take derivatives?

* [Symbolic differentiation](http://tutorial.math.lamar.edu/pdf/common_derivatives_integrals.pdf)
* [Numerical differentiation](https://en.wikipedia.org/wiki/Finite_difference) (the method of finite differences)
* [Automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation)

Take the derivative of $f(x) = x^2$.

Symbolic derivative $f'(x) = rx^{r-1}$

$f(4) = 4^2 = 16$

$f'(4) = 2 \cdot 4 = 8$

This can be done in TensorFlow:

In [ ]:
x = tf.constant(4.0)

with tf.GradientTape() as t:
  t.watch(x)
  z = tf.multiply(x, x)

# Derivative of z with respect to the original input tensor x
dz_dx = t.gradient(z, x)
print(dz_dx)

Lets express the [Logistic function](https://en.wikipedia.org/wiki/Logistic_function) in TensorFlow. This is also called the Sigmoid Activation function in neural network literature.

$f(x) = \frac{1}{1 + e^{-x}}$

Written in TensorFlow:

In [ ]:
x = tf.constant([5.0])
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.divide(1.0,tf.add(1,tf.exp(tf.negative(x))))
    
print(y)
dy_dx = t.gradient(y, x)
print(dy_dx)

Lets check the regular function.

In [ ]:
import math

1/(1+math.exp(-5))

And lets check the derivative:
    
$f'(x) = \frac{e^x}{(e^x + 1)^2}$


In [ ]:
math.exp(-5)/(math.exp(-5)+1)**2

In [ ]:
x = tf.ones((2, 2))
y = tf.reduce_sum(x)
z = tf.multiply(y, y)
y.numpy()

How to take second (and beyond) derivatives:

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  with tf.GradientTape() as gg:
    gg.watch(x)
    y = x * x
  dy_dx = gg.gradient(y, x)     # Will compute to 6.0
d2y_dx2 = g.gradient(dy_dx, x)  # Will compute to 2.0

# Custom Loss (Objective) Function

$ \operatorname{RMSE}=\sqrt{\frac{\sum_{t=1}^T (\hat y_t - y_t)^2}{T}} $

In [ ]:
def mean_pred(y_true, y_pred):
    return tf.sqrt(tf.divide(tf.reduce_sum(tf.pow(tf.subtract(y_true, y_pred),2.0)),tf.cast(tf.size(y_true), tf.float32)))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss=mean_pred, optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

# Custom Activation (Transfer) Functions

In [ ]:
import tensorflow as tf
def elliot_sym(x):
    return tf.divide(x,tf.add(1.0,tf.abs(x)))     

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Build the neural network
sgd = tf.keras.optimizers.SGD(lr=1e-10, decay=1e-6, momentum=0.9, nesterov=True)
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation=elliot_sym)) # Hidden 1
model.add(Dense(10, activation=elliot_sym)) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=400)